In [46]:
import sqlite3, pandas, datetime
conn = sqlite3.connect('monkepo.db')
curs = conn.cursor()
curs.execute("CREATE TABLE Monkepo(\
            mid INTEGER PRIMARY KEY AUTOINCREMENT,\
            name TEXT,\
            UNIQUE(name))")
curs.execute("CREATE TABLE Appearance(\
            mid INTEGER,\
            latitude REAL,\
            longitude REAL,\
            datetime DATETIME,\
            FOREIGN KEY(mid) REFERENCES Monkepo(mid))")
curs.execute("CREATE TABLE Class(\
            mid INTEGER,\
            major BOOLEAN,\
            type TEXT,\
            PRIMARY KEY(mid, major),\
            FOREIGN KEY(mid) REFERENCES Monkepo(mid))")
monkcsv = pandas.read_csv('monkepo.csv')
monk_names = monkcsv.name.drop_duplicates()
monk_names.to_sql('Monkepo', conn, if_exists='append', index=False)


def converttime(row):
    try:
        return datetime.datetime.combine(
            datetime.datetime.strptime(row['date'], '%Y-%m-%d'),
            datetime.datetime.time(datetime.datetime.strptime(row['time'], '%H:%M:%S.%f')))
    except ValueError:
        return datetime.datetime.combine(
            datetime.datetime.strptime(row['date'], '%Y-%m-%d'),
            datetime.datetime.time(datetime.datetime.strptime(row['time'], '%H:%M:%S')))


def monkepotomid(row):
    return pandas.read_sql_query(
        'SELECT mid from Monkepo M WHERE M.name = "'+ row['name'] + '"', conn)['mid'][0]


monkcsv['mid'] = monkcsv.apply(monkepotomid, axis=1)
monk_sightings = monkcsv.drop(['majorclass', 'minorclass', 'date', 'time', 'name'], axis=1)
monk_sightings['datetime'] = monkcsv.apply(converttime, axis=1)
monk_sightings.to_sql('Appearance', conn, if_exists='append', index=False)
monkcsv = monkcsv.drop_duplicates()
monk_class_major = monkcsv.drop(
    ['longitude', 'latitude', 'name', 'date', 'time', 'minorclass'], axis=1)
monk_class_minor = monkcsv.drop(
    ['longitude', 'latitude', 'name', 'date', 'time', 'majorclass'], axis=1)
monk_class_minor = monk_class_minor.drop_duplicates()
monk_class_major = monk_class_major.drop_duplicates()
monk_class_major.rename(columns={'majorclass':'type'}, inplace=True)
monk_class_minor.rename(columns={'minorclass':'type'}, inplace=True)
monk_class_major['major'] = True
monk_class_minor['major'] = False
monk_class = pandas.concat([monk_class_major, monk_class_minor])
monk_class = monk_class[monk_class.type != 'None']
monk_class.to_sql('Class', conn, if_exists='append', index=False)
conn.commit()
conn.close()

In [72]:
def maxminclass(db):
    classes = pandas.read_sql_query('SELECT DISTINCT type FROM Class', sqlite3.connect(db))
    res = []
    for i in classes['type']:
        maxmonk = pandas.read_sql_query('SELECT name\
                                 FROM (Monkepo JOIN Appearance USING(mid)\
                                       JOIN Class USING(mid)) AS M\
                                 WHERE type = "' + i + '"\
                                 GROUP BY M.mid\
                                 ORDER BY COUNT(M.mid) DESC\
                                 LIMIT 1',
                                 sqlite3.connect(db))
        minmonk = pandas.read_sql_query('SELECT name\
                                 FROM (Monkepo JOIN Appearance USING(mid)\
                                       JOIN Class USING(mid)) AS M\
                                 WHERE type = "' + i + '"\
                                 GROUP BY M.mid\
                                 ORDER BY COUNT(M.mid) ASC\
                                 LIMIT 1',
                                 sqlite3.connect(db))
        res.append((i, maxmonk.name[0], minmonk.name[0]))
    return res
         
    
%timeit[maxminclass('monkepo.db')]
    

1 loop, best of 3: 10 s per loop


In [80]:
conn = sqlite3.connect('monkepo.db')
curs = conn.cursor()
curs.execute('CREATE TABLE SupplyStations(\
            stationID INTEGER PRIMARY KEY,\
            latitude REAL,\
            longitude REAL)')
pandas.read_csv("stations.csv").to_sql('SupplyStations', conn, if_exists='append',
                                                  index=False)
conn.commit()
conn.close()



OperationalError: table SupplyStations already exists

In [40]:
import numpy, pandas, sqlite3


def stationpopularity(db):
    appearances = pandas.read_sql_query('SELECT A.latitude, A.longitude\
                                        FROM Appearance A',
                                        sqlite3.connect(db))
    stations = pandas.read_sql_query('SELECT S.stationID, S.latitude, S.longitude\
                                            FROM SupplyStations S',
                                            sqlite3.connect(db))
    stations['count'] = 0
    for applat, applong in appearances.values:
        min_dist = 99999999999999999
        count = 0
        station = 0
        for statid, statlat, statlong in stations.values:
            dist = numpy.sqrt(numpy.power(applat-statlat, 2)+numpy.power(applong-statlong, 2))
            if dist < min_dist:
                min_dist = dist
                station = count
            count += 1
        stations.set_value(station, 'count', stations['count'][station] + 1)
    stations.sort_values('count', axis=0, ascending=False, inplace=True, kind='quicksort')
    res = []
    for statid, statlat, statlong, count in stations.values:
        res.append((statid, statlat, statlong, count))
    return res
        

stationpopularity('monkepo.db')

[(1947.0, 35.770122999999998, -78.631609999999995, 115948.0),
 (1739.0, 36.095616, -79.266743000000005, 0.0),
 (1745.0, 36.094323000000003, -79.271895000000001, 0.0),
 (1744.0, 36.092072000000002, -79.268502999999995, 0.0),
 (1743.0, 36.095298, -79.266994999999994, 0.0),
 (1742.0, 36.096445000000003, -79.267358999999999, 0.0),
 (1741.0, 36.094836999999998, -79.265162000000004, 0.0),
 (1740.0, 36.096055999999997, -79.266388000000006, 0.0),
 (1000.0, 35.897835000000001, -79.18871, 0.0),
 (1747.0, 36.093651000000001, -79.271197999999998, 0.0),
 (1738.0, 36.095039, -79.263670000000005, 0.0),
 (1737.0, 36.099507000000003, -79.266267999999997, 0.0),
 (1736.0, 36.096716000000001, -79.273238000000006, 0.0),
 (1735.0, 36.096722, -79.271977000000007, 0.0),
 (1734.0, 36.096592000000001, -79.271726000000001, 0.0),
 (1733.0, 36.097126000000003, -79.272442999999996, 0.0),
 (1746.0, 36.094082999999998, -79.270697999999996, 0.0),
 (1748.0, 36.095516000000003, -79.274522000000005, 0.0),
 (1731.0, 36.09

In [20]:
import pandas, sqlite3
def appearancefreqpy(db):
    monks = pandas.read_sql_query('SELECT M.name\
                                  FROM (Monkepo JOIN Appearance USING(mid)) AS M',
                                  sqlite3.connect(db))
    monkviews = {}
    for row in monks.values:
        monkviews[row[0]] = monkviews.get(row[0],0) + 1
    print(len(monkviews))
    return sorted(monkviews.items(), key=lambda tup: tup[1], reverse=True)

pandas.DataFrame(appearancefreqpy('monkepo.db'), columns=['Name', 'Appearances'])

123


,Name,Appearances
0,PigDye,24446
1,AttaRat,18662
2,RopeSaw,10397
3,DewEel,8717
4,Veeee,7356
5,NovaNet,4767
6,TarPiece,4036
7,TubaZ,3641
8,ARasp,2593
9,AnIronAd-,2497
